# Data Augmentation

## 1. Text preprocessing

### 1.1. 맞춤법 처리 여부
hanspell의 spell_checker를 돌리게 되면 많은 시간이 소모되는데 (`train.csv`에 대해 약 10분), 

실질적으로 얼마나 많은 단어들이 교정되는지부터 보자.

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
from tqdm import tqdm

In [2]:
train_path = './data/train.csv'
dev_path = './data/dev.csv'
test_path = './data/test.csv'

먼저 특수문자를 제거하겠다. 

In [18]:
train_df = pd.read_csv(train_path)
dev_df = pd.read_csv(dev_path)
test_df = pd.read_csv(test_path)

In [98]:
def remove_special_char(df):
    def _remove_special_char(sentence):
        p = re.compile("""[^A-Za-zㄱ-ㅎ가-힣0-9,.?!]""")
        return p.sub(" ", sentence)
    df.sentence_1 = df.sentence_1.map(_remove_special_char)
    df.sentence_2 = df.sentence_2.map(_remove_special_char)
    return df


In [20]:
train_df = remove_special_char(train_df)

In [3]:
from hanspell import spell_checker
import re

In [4]:
p = re.compile('[^A-Za-zㄱ-ㅎ가-힣0-9.?,!:!]')
A = p.sub(" ","%안녕하세요,감사해요?잘있어요!다시만나요. :>")
A

' 안녕하세요,감사해요?잘있어요!다시만나요.   '

In [21]:
error_cnt = []
for i in tqdm(range(len(train_df))):
    temp = spell_checker.check(train_df['sentence_1'].iloc[i]).as_dict()
    err = temp['errors']
    error_cnt.append(err)

100%|██████████| 9324/9324 [04:04<00:00, 38.15it/s]


In [22]:
np.mean(error_cnt)

1.0084727584727584

위의 결과에서 확인할 수 있듯이, 각 문장별로 평균 1개 미만으로 맞춤법이 교정되기 때문에, 맞춤법 교정은 필요없다고 판단된다.

In [23]:
error_cnt2 = []
for i in tqdm(range(len(train_df))):
    temp = spell_checker.check(train_df['sentence_2'].iloc[i]).as_dict()
    err = temp['errors']
    error_cnt2.append(err)

100%|██████████| 9324/9324 [03:50<00:00, 40.47it/s]


In [24]:
np.mean(error_cnt2)

0.7360574860574861

`sentence_2`에 대해서도 마찬가지이다. 따라서, hanspell은 사용하지 않아도 되겠다.

또한, 특수문자에 대해서도 생각해야 하는 게, 가령 `'6:30'` 과 `'6시 30분'`은 같은 뜻이다.

특수문자를 제거해버릴 경우, `6 30` 또는 `630`으로 바뀌어버린다면 원래 텍스트의 의미가 왜곡될 수 있다. 

따라서, 특수문자 제거도 하지 않는 게 좋다고 판단된다.

다행히 Huggingface의 tokenizer가 특수문자에 대해서도 처리해준다. (아래 코드 참고)

In [56]:
from transformers import AutoTokenizer
model_name = "klue/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer

BertTokenizerFast(name_or_path='klue/roberta-base', vocab_size=32000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [43]:
tokenizer("%")

{'input_ids': [0, 9, 2], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}

In [36]:
tokenizer.convert_ids_to_tokens(9)

'%'

단, 텍스트 데이터에 있는 `<PERSON>`의 경우 token에 추가로 넣어주면 좋을 것 같다. 

이유는 `<PERSON>`이 사람의 이름을 대체하고 있고, 본래 영단어의 뜻인 사람으로서 쓰이지 않기 때문이다.

In [57]:
len(tokenizer)

32000

In [58]:
tokenizer.add_tokens(["<PERSON>"])

1

In [59]:
tokenizer.convert_ids_to_tokens(32000)

'<PERSON>'

In [60]:
# 단어가 증가한 것을 볼 수 있다.
len(tokenizer)

32001

위와 같이 token을 추가해준 경우, model의 token_embedding 크기를 조절해야 한다.

In [61]:
# model.resize_token_embeddings(len(tokenizer))

## 2. Data augmentation

### 2.1. `sentence_1` - `sentence_2` swapping

BERT 기반 모델이 bidirectional하다고는 하지만, NSP같은 Pre-training task를 생각해보면 
앞뒤의 문장 순서가 바뀐다면 다른 결과가 나올 수 있다. (Segment embedding 도 있고... Positional encoding도 있고..)

문장 순서가 뒤바뀌더라도 같은 결과를 주어야 한다는 것으로부터 `sentence_1`과 `sentence_2`가 바뀐 데이터를 추가해주면 
문장 간의 유사도 측정 task에 더 좋은 학습을 할 수 있을 것 같다.

In [114]:
train_df = pd.read_csv(train_path)
dev_df = pd.read_csv(dev_path)
test_df = pd.read_csv(test_path)

In [115]:
# 문장 내에서 다른 단어들과 함께 의미가 있을 수 있는 특수문자 @, ?, ... 를 제외한 특수문자를 제거.
# 예) \ (백슬래쉬)
def remove_special_char(df):
    def _remove_special_char(sentence):
        p = re.compile("""[^A-Za-zㄱ-ㅎ가-힣0-9,<.>?/;:'"!@%^&*()\[\]\{\}+=-_]""")
        return p.sub(" ", sentence)
    df.sentence_1 = df.sentence_1.map(_remove_special_char)
    df.sentence_2 = df.sentence_2.map(_remove_special_char)
    return df


In [116]:
train_df = remove_special_char(train_df)
dev_df = remove_special_char(dev_df)
test_df = remove_special_char(test_df)

In [117]:
temp = train_df.copy()
cols = list(temp.columns)
cols

['id', 'source', 'sentence_1', 'sentence_2', 'label', 'binary-label']

In [118]:
temp.rename(columns={'sentence_1':'sentence_2', 'sentence_2':'sentence_1'}, inplace=True)
temp = temp[cols]
temp

,id,source,sentence_1,sentence_2,label,binary-label
0,boostcamp-sts-v1-train-000,nsmc-sampled,"반전도 있고,사랑도 있고재미도있네요.",스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요,2.2,0.0
1,boostcamp-sts-v1-train-001,slack-rtt,"오, 액세스 권한이 없다고 합니다.",앗 제가 접근권한이 없다고 뜹니다;;,4.2,1.0
2,boostcamp-sts-v1-train-002,petition-sampled,주택청약 무주택기준 변경해주세요.,주택청약조건 변경해주세요.,2.4,0.0
3,boostcamp-sts-v1-train-003,slack-sampled,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,입사후 처음 대면으로 만나 반가웠습니다.,3.0,1.0
4,boostcamp-sts-v1-train-004,slack-sampled,꼬옥 실제로 한번 뵈어요 뿌뿌뿌 ! !,뿌듯뿌듯 하네요!!,0.0,0.0
...,...,...,...,...,...,...
9319,boostcamp-sts-v1-train-9319,petition-sampled,본인이 납부한 국민연금 금액을 기준으로 대출을 받을 수 있는 제도를 마련해 주세요,교원능력개발평가에서 교원이 보호받을 수 있는 장치를 마련해야합니다,0.2,0.0
9320,boostcamp-sts-v1-train-9320,petition-sampled,여성가족부 폐지를 청원 합니다.,여성가족부의 폐지를 원합니드,4.2,1.0
9321,boostcamp-sts-v1-train-9321,petition-sampled,공무원 봉급좀 줄이지좀 마세요,국회의원들 월급좀 줄여주세요,0.6,0.0
9322,boostcamp-sts-v1-train-9322,slack-sampled,오늘 못먹은 밥은 꼭 담에 먹기로 하고요!!,오늘 못한 점심은 다음에 다시 츄라이 하기로 해요!!,3.2,1.0


In [119]:
train_df = pd.concat([train_df, temp])
train_df

,id,source,sentence_1,sentence_2,label,binary-label
0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요,"반전도 있고,사랑도 있고재미도있네요.",2.2,0.0
1,boostcamp-sts-v1-train-001,slack-rtt,앗 제가 접근권한이 없다고 뜹니다;;,"오, 액세스 권한이 없다고 합니다.",4.2,1.0
2,boostcamp-sts-v1-train-002,petition-sampled,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,2.4,0.0
3,boostcamp-sts-v1-train-003,slack-sampled,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,3.0,1.0
4,boostcamp-sts-v1-train-004,slack-sampled,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌 ! !,0.0,0.0
...,...,...,...,...,...,...
9319,boostcamp-sts-v1-train-9319,petition-sampled,본인이 납부한 국민연금 금액을 기준으로 대출을 받을 수 있는 제도를 마련해 주세요,교원능력개발평가에서 교원이 보호받을 수 있는 장치를 마련해야합니다,0.2,0.0
9320,boostcamp-sts-v1-train-9320,petition-sampled,여성가족부 폐지를 청원 합니다.,여성가족부의 폐지를 원합니드,4.2,1.0
9321,boostcamp-sts-v1-train-9321,petition-sampled,공무원 봉급좀 줄이지좀 마세요,국회의원들 월급좀 줄여주세요,0.6,0.0
9322,boostcamp-sts-v1-train-9322,slack-sampled,오늘 못먹은 밥은 꼭 담에 먹기로 하고요!!,오늘 못한 점심은 다음에 다시 츄라이 하기로 해요!!,3.2,1.0


In [120]:
train_df.describe(include='all')

,id,source,sentence_1,sentence_2,label,binary-label
count,18648,18648,18648,18648,18648.000000,18648.000000
unique,9324,6,18634,18634,NaN,NaN
top,boostcamp-sts-v1-train-000,petition-sampled,다음에 만나요!,청소년보호법 폐지해주세요,NaN,NaN
freq,2,4704,2,2,NaN,NaN
mean,NaN,NaN,NaN,NaN,1.849968,0.389532
std,NaN,NaN,NaN,NaN,1.602169,0.487657
min,NaN,NaN,NaN,NaN,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,0.200000,0.000000
50%,NaN,NaN,NaN,NaN,1.600000,0.000000
75%,NaN,NaN,NaN,NaN,3.400000,1.000000


1차적으로 증강된 데이터셋을 `new_data_1` 폴더에 저장하겠다.

In [121]:
# if not os.path.isdir("./new_data_1"):
#     os.makedirs("./new_data_1")
# train_df.to_csv("./new_data_1/train.csv")
# dev_df.to_csv("./new_data_1/dev.csv")
# test_df.to_csv("./new_data_1/test.csv")

### 2.2. 같은 문장 생성 (self-translation?)
고 점수의 데이터가 부족하기 때문에 binary-label이 0인 데이터들을 대상으로 `sentence_1 == sentence_2` 데이터를 생성해주면 
낮은 점수의 데이터들에 대해서 유사도가 높은 문장에 대한 정보를 줄 수 있기 때문에 더 효과적인 학습이 이루어질 것으로 추측된다.

특히 기존의 label의 skewed distribution이 bimodal distribution으로 바뀌게 되는 효과가 있다. 
물론 이렇게 하는 게 더 좋을지는... 해봐야 알 듯.

In [122]:
temp = train_df[train_df['binary-label'] == 0.0].copy()

In [124]:
temp

,id,source,sentence_1,sentence_2,label,binary-label
0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요,"반전도 있고,사랑도 있고재미도있네요.",2.2,0.0
2,boostcamp-sts-v1-train-002,petition-sampled,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,2.4,0.0
4,boostcamp-sts-v1-train-004,slack-sampled,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌 ! !,0.0,0.0
7,boostcamp-sts-v1-train-007,nsmc-sampled,이렇게 귀여운 쥐들은 처음이네요.ㅎㅎㅎ,이렇게 지겨운 공포영화는 처음..,0.6,0.0
8,boostcamp-sts-v1-train-008,petition-sampled,미세먼지 해결이 가장 시급한 문제입니다!,가장 시급한 것이 신생아실 관리입니다!!!,0.4,0.0
...,...,...,...,...,...,...
9316,boostcamp-sts-v1-train-9316,slack-sampled,너무 맛있어서 또 가고싶은집!!,어쩌다 보니 사람 들어간 사진은 못찍었는데 너무 맛있었고 재밌었습니다!!,1.6,0.0
9317,boostcamp-sts-v1-train-9317,petition-sampled,동물원법을 강화시켜 주세요,동물보호법을 강화해 주세요,1.5,0.0
9319,boostcamp-sts-v1-train-9319,petition-sampled,본인이 납부한 국민연금 금액을 기준으로 대출을 받을 수 있는 제도를 마련해 주세요,교원능력개발평가에서 교원이 보호받을 수 있는 장치를 마련해야합니다,0.2,0.0
9321,boostcamp-sts-v1-train-9321,petition-sampled,공무원 봉급좀 줄이지좀 마세요,국회의원들 월급좀 줄여주세요,0.6,0.0


In [127]:
temp_len = len(temp)
rest_len = len(train_df) - len(temp)
rest_len

7264

In [154]:
import random
random.seed(777)

out = random.sample(range(temp_len), temp_len - rest_len)


temp_df = temp.iloc[out]
temp_df['sentence_2'] = temp_df['sentence_1']
temp_df['label'] = 5.0
temp_df['binary-label'] = 1.0
temp_df

/tmp/ipykernel_17993/813168914.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['sentence_2'] = temp_df['sentence_1']
/tmp/ipykernel_17993/813168914.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['label'] = 5.0
/tmp/ipykernel_17993/813168914.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

,id,source,sentence_1,sentence_2,label,binary-label
6134,boostcamp-sts-v1-train-6134,slack-sampled,와 대박 무슨 엽서사진같아요!!,와 대박 무슨 엽서사진같아요!!,5.0,1.0
2670,boostcamp-sts-v1-train-2670,slack-sampled,매일 보는 저희 CE 팀의 <PERSON> 님과 이야기를 나눴습니다 !,매일 보는 저희 CE 팀의 <PERSON> 님과 이야기를 나눴습니다 !,5.0,1.0
2665,boostcamp-sts-v1-train-2665,petition-sampled,교육학 중등 임 용고사 객관식으로 바꿔주세요,교육학 중등 임 용고사 객관식으로 바꿔주세요,5.0,1.0
572,boostcamp-sts-v1-train-572,slack-rtt,(게시물에 약간의 변경이 있을 수 있습니다),(게시물에 약간의 변경이 있을 수 있습니다),5.0,1.0
6094,boostcamp-sts-v1-train-6094,petition-sampled,노인 고령화와 일자리문제,노인 고령화와 일자리문제,5.0,1.0
...,...,...,...,...,...,...
1224,boostcamp-sts-v1-train-1224,slack-sampled,중간만 해도 45점이네요.,중간만 해도 45점이네요.,5.0,1.0
9137,boostcamp-sts-v1-train-9137,nsmc-sampled,와! 이제 진짜 욕나온다..,와! 이제 진짜 욕나온다..,5.0,1.0
3126,boostcamp-sts-v1-train-3126,petition-rtt,이언주의원 공부좀 합시다,이언주의원 공부좀 합시다,5.0,1.0
1502,boostcamp-sts-v1-train-1502,slack-sampled,"9번째, <PERSON> 님과 만났습니다.","9번째, <PERSON> 님과 만났습니다.",5.0,1.0


In [155]:
ntrain_df = pd.concat([train_df, temp_df])
ntrain_df

,id,source,sentence_1,sentence_2,label,binary-label
0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요,"반전도 있고,사랑도 있고재미도있네요.",2.2,0.0
1,boostcamp-sts-v1-train-001,slack-rtt,앗 제가 접근권한이 없다고 뜹니다;;,"오, 액세스 권한이 없다고 합니다.",4.2,1.0
2,boostcamp-sts-v1-train-002,petition-sampled,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,2.4,0.0
3,boostcamp-sts-v1-train-003,slack-sampled,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,3.0,1.0
4,boostcamp-sts-v1-train-004,slack-sampled,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌 ! !,0.0,0.0
...,...,...,...,...,...,...
1224,boostcamp-sts-v1-train-1224,slack-sampled,중간만 해도 45점이네요.,중간만 해도 45점이네요.,5.0,1.0
9137,boostcamp-sts-v1-train-9137,nsmc-sampled,와! 이제 진짜 욕나온다..,와! 이제 진짜 욕나온다..,5.0,1.0
3126,boostcamp-sts-v1-train-3126,petition-rtt,이언주의원 공부좀 합시다,이언주의원 공부좀 합시다,5.0,1.0
1502,boostcamp-sts-v1-train-1502,slack-sampled,"9번째, <PERSON> 님과 만났습니다.","9번째, <PERSON> 님과 만났습니다.",5.0,1.0


In [157]:
# if not os.path.isdir("./new_data_2"):
#     os.makedirs("./new_data_2")
# ntrain_df.to_csv("./new_data_2/train.csv")
# dev_df.to_csv("./new_data_2/dev.csv")
# test_df.to_csv("./new_data_2/test.csv")

### 2.3. round-trip translation with backtranslate
구글 번역 API를 활용하는 패키지 roundtrip을 사용해서 번역기를 거친 텍스트를 생성하려고 한다. 

단, 데이터가 많은 0.0점과 두 문장이 같은 5.0 점은 제외한다. 

또한 번역기를 거친 rtt 텍스트도 무시한다.

In [158]:
!pip install roundtrip

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.4/128.4 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.3/120.3 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.2/178.2 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 76.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [200]:
temp = ntrain_df[(ntrain_df['label'] < 5.0) & (ntrain_df['label'] > 0.0) &  ~(ntrain_df['source'].str.contains("rtt"))].copy()
temp
# temp = ntrain_df[(ntrain_df['label'] >= 4.0) | (ntrain_df['label'] < 0.5)].copy()
# temp

,id,source,sentence_1,sentence_2,label,binary-label
0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요,"반전도 있고,사랑도 있고재미도있네요.",2.2,0.0
2,boostcamp-sts-v1-train-002,petition-sampled,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,2.4,0.0
3,boostcamp-sts-v1-train-003,slack-sampled,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,3.0,1.0
7,boostcamp-sts-v1-train-007,nsmc-sampled,이렇게 귀여운 쥐들은 처음이네요.ㅎㅎㅎ,이렇게 지겨운 공포영화는 처음..,0.6,0.0
8,boostcamp-sts-v1-train-008,petition-sampled,미세먼지 해결이 가장 시급한 문제입니다!,가장 시급한 것이 신생아실 관리입니다!!!,0.4,0.0
...,...,...,...,...,...,...
9319,boostcamp-sts-v1-train-9319,petition-sampled,본인이 납부한 국민연금 금액을 기준으로 대출을 받을 수 있는 제도를 마련해 주세요,교원능력개발평가에서 교원이 보호받을 수 있는 장치를 마련해야합니다,0.2,0.0
9320,boostcamp-sts-v1-train-9320,petition-sampled,여성가족부 폐지를 청원 합니다.,여성가족부의 폐지를 원합니드,4.2,1.0
9321,boostcamp-sts-v1-train-9321,petition-sampled,공무원 봉급좀 줄이지좀 마세요,국회의원들 월급좀 줄여주세요,0.6,0.0
9322,boostcamp-sts-v1-train-9322,slack-sampled,오늘 못먹은 밥은 꼭 담에 먹기로 하고요!!,오늘 못한 점심은 다음에 다시 츄라이 하기로 해요!!,3.2,1.0


In [201]:
import roundtrip

from functools import partial
btt_func = partial(roundtrip.backtranslate, to_language="en", from_language="ko")

temp['sentence_1'] = temp['sentence_1'].map(btt_func)

,id,source,sentence_1,sentence_2,label,binary-label
4773,boostcamp-sts-v1-train-4773,petition-sampled,100번째 커리큘럼 없애기,100대 교육과정 없애주세요,2.2,0.0
8719,boostcamp-sts-v1-train-8719,petition-sampled,인천 부천 중고차 허위매물,인천부천지역 중고차 허위매물,2.2,0.0
7642,boostcamp-sts-v1-train-7642,slack-rtt,이것을 켜고 점심을 먹으십시오!,이것을 켜고 점심을 먹으십시오!,3.2,1.0
304,boostcamp-sts-v1-train-304,petition-sampled,가상 화폐 폐지 반대 / 가상 정부 폐지 찬성,가상화폐 폐지 반대/가상정부 폐지 찬성,1.2,0.0
1596,boostcamp-sts-v1-train-1596,petition-sampled,용산전자상가 거품폭주,용산 전자상가 거품가격 폭주,1.8,0.0
2812,boostcamp-sts-v1-train-2812,slack-rtt,커뮤니케이션 스터디 지원,커뮤니케이션 스터디 지원,2.6,1.0
4644,boostcamp-sts-v1-train-4644,petition-sampled,문재인 정부와 지상파에 바란다,문재인 정부와 지상파방송사에 바란다,0.6,0.0
5284,boostcamp-sts-v1-train-5284,slack-sampled,제주도에서 좋은 시간 보내시고 건강하게 돌아오세요,제주도에서 좋은 시간 보내다 건강히 돌아오셔요,0.6,0.0
7720,boostcamp-sts-v1-train-7720,nsmc-rtt,장르 불문. 시간에 관계없이. 공간에 상관없이. 당신이 좋아하는 것을하십시오!,장르 불문. 시간 불문. 공간 불문. 네 맘대로 해라!,2.8,1.0
784,boostcamp-sts-v1-train-784,slack-sampled,주차도 좋고 직원들 매너도 좋아요!,주차도 좋고 직원들 매너도 좋고요!,0.4,0.0


## 3. 최종 작업 용 데이터셋 생성

위의 세 가지 작업을 바탕으로 DA 처리한 데이터셋을 만들어보겠다.

작업을 거친 후 총 4개의 데이터셋을 만들게 된다.
- `./data/` : raw data
- `./data_1/` : RTT
- `./data_2/` : RTT + SWAP
- `./data_3/` : RTT + SWAP + Self-translation

In [213]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm

In [214]:
train_path = './data/train.csv'
dev_path = './data/dev.csv'
test_path = './data/test.csv'

In [215]:
train_df = pd.read_csv(train_path)
dev_df = pd.read_csv(dev_path)
test_df = pd.read_csv(test_path)

In [217]:
# 특수문자부터 제거
def remove_special_char(df):
    def _remove_special_char(sentence):
        p = re.compile("""[^A-Za-zㄱ-ㅎ가-힣0-9,<.>?/;:'"!@%^&*()\[\]\{\}+=-_]""")
        return p.sub(" ", sentence)
    df.sentence_1 = df.sentence_1.map(_remove_special_char)
    df.sentence_2 = df.sentence_2.map(_remove_special_char)
    return df

In [218]:
train_df = remove_special_char(train_df)
dev_df = remove_special_char(dev_df)
test_df = remove_special_char(test_df)

In [219]:
# RTT : source가 rtt가 아닌 데이터를 기준으로 `label` 점수가 0.0, 5.0이 아닌 데이터에 대해서 sentence_1을 rtt 문장으로 변환.
import roundtrip

from functools import partial
btt_func = partial(roundtrip.backtranslate, to_language="en", from_language="ko")

temp1 = train_df[(train_df['label'] < 5.0) & (train_df['label'] > 0.0) &  ~(train_df['source'].str.contains("rtt"))].copy()
temp2 = train_df[(train_df['label'] < 5.0) & (train_df['label'] > 0.0) &  ~(train_df['source'].str.contains("rtt"))].copy()
temp1

,id,source,sentence_1,sentence_2,label,binary-label
0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요,"반전도 있고,사랑도 있고재미도있네요.",2.2,0.0
2,boostcamp-sts-v1-train-002,petition-sampled,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,2.4,0.0
3,boostcamp-sts-v1-train-003,slack-sampled,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,3.0,1.0
7,boostcamp-sts-v1-train-007,nsmc-sampled,이렇게 귀여운 쥐들은 처음이네요.ㅎㅎㅎ,이렇게 지겨운 공포영화는 처음..,0.6,0.0
8,boostcamp-sts-v1-train-008,petition-sampled,미세먼지 해결이 가장 시급한 문제입니다!,가장 시급한 것이 신생아실 관리입니다!!!,0.4,0.0
...,...,...,...,...,...,...
9319,boostcamp-sts-v1-train-9319,petition-sampled,교원능력개발평가에서 교원이 보호받을 수 있는 장치를 마련해야합니다,본인이 납부한 국민연금 금액을 기준으로 대출을 받을 수 있는 제도를 마련해 주세요,0.2,0.0
9320,boostcamp-sts-v1-train-9320,petition-sampled,여성가족부의 폐지를 원합니드,여성가족부 폐지를 청원 합니다.,4.2,1.0
9321,boostcamp-sts-v1-train-9321,petition-sampled,국회의원들 월급좀 줄여주세요,공무원 봉급좀 줄이지좀 마세요,0.6,0.0
9322,boostcamp-sts-v1-train-9322,slack-sampled,오늘 못한 점심은 다음에 다시 츄라이 하기로 해요!!,오늘 못먹은 밥은 꼭 담에 먹기로 하고요!!,3.2,1.0


In [220]:
import time
start = time.time()
# temp1 : sentence_1에 대한 rtt
temp1['sentence_1'] = temp1['sentence_1'].map(btt_func)
print(time.time() - start)
start = time.time()
# temp2 : sentence_2에 대한 rtt
temp2['sentence_2'] = temp2['sentence_2'].map(btt_func)
print(time.time() - start)

KeyboardInterrupt: 

In [ ]:
train1_df = pd.concat([train_df, temp1, temp2])
train1_df = train1_df.drop_duplicates(subset=['sentence_1','sentence_2'])

In [ ]:
if not os.path.isdir("./data_1"):
    os.makedirs("./data_1")
train1_df.to_csv("./data_1/train.csv")
dev_df.to_csv("./data_1/dev.csv")
test_df.to_csv("./data_1/test.csv")

In [ ]:
## swap
temp = train1_df.copy()
cols = list(temp.columns)
temp.rename(columns={'sentence_1':'sentence_2', 'sentence_2':'sentence_1'}, inplace=True)
temp = temp[cols]
temp

In [ ]:
train2_df = pd.concat([train1_df, temp])

In [ ]:
if not os.path.isdir("./data_2"):
    os.makedirs("./data_2")
train2_df.to_csv("./data_2/train.csv")
dev_df.to_csv("./data_2/dev.csv")
test_df.to_csv("./data_2/test.csv")

In [ ]:
## sentence 1 복제
temp3 = train2_df[train2_df['binary-label'] == 0.0].copy()
temp_len = len(temp3)
rest_len = len(train2_df) - len(temp)
rest_len

In [ ]:
import random
random.seed(777)

out = random.sample(range(temp_len), temp_len - rest_len)

temp_df = temp3.iloc[out]
temp_df['sentence_2'] = temp_df['sentence_1']
temp_df['label'] = 5.0
temp_df['binary-label'] = 1.0
temp_df

In [ ]:
train3_df = pd.concat([train2_df, temp_df])
train3_df

In [ ]:
if not os.path.isdir("./data_3"):
    os.makedirs("./data_3")
train3_df.to_csv("./data_3/train.csv")
dev_df.to_csv("./data_3/dev.csv")
test_df.to_csv("./data_3/test.csv")